## Import libs and custom vars

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from libs.utils import find_penalty, return_outcome, check_element, add_labels, find_range
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import datetime
from numba import jit

<b>L'obbiettivo del progetto è la costruzione di un modello per predizioni di match internazionali</b>


<ul>
    <li>Ci sono alcuni nomi di squadre che sono cambiate nel tempo, uniformizza </li>
    <li>Shootouts è il db dei penalty, quindi bisogna manipolare il dataset per vedere chi vince alla fine quella partita</li>
</ul>

### Descrizione del dataset


Il Dataset contiene 42483 risultati di partite di calcio internazionale maschile. In particolare, sono incluse sia le partite ufficiali di tutte le competizioni: dalle amichevoli alla coppa del mondo eccetto i Giochi Olimpici o le partite dove almeno una delle due squadre era la squadra B o U-23. Inoltre, il dataset presenta un ulteriore file che contiene 432 risultati di partite terminate con i calci di rigore, <u>che non considereremo in quanto il modello dovrà predirre il risultato come: 'Home win', 'Draw', 'Away win'.</u>

In [3]:
dst_df = pd.read_csv('dataset/results.csv');

print("Null values:", dst_df.isnull().values.any(), "| shape:", dst_df.shape)
print("***"*15)
print(dst_df.info())

Null values: False | shape: (42483, 9)
*********************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42483 entries, 0 to 42482
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        42483 non-null  object
 1   home_team   42483 non-null  object
 2   away_team   42483 non-null  object
 3   home_score  42483 non-null  int64 
 4   away_score  42483 non-null  int64 
 5   tournament  42483 non-null  object
 6   city        42483 non-null  object
 7   country     42483 non-null  object
 8   neutral     42483 non-null  bool  
dtypes: bool(1), int64(2), object(6)
memory usage: 2.6+ MB
None


### Preparazione del daset
- campo date formattato in anno
- colonna 'risultato'
- converto home_score e away_score in float

In [4]:
dst_df = dst_df.astype({"home_score": float, "away_score": float})

dst_df['outcome'] = None
dst_df['day'] = None
dst_df['month'] = None
dst_df['year'] = None

date_separator = ', '

counter = 0
for i in range(0, len(dst_df)):
    counter = counter + 1
    print('*** Processing %d/%d ***' % (counter, len(dst_df)), end='\r')

    # estraggo dati necessari dalla data
    row_date = datetime.date.fromisoformat(dst_df.iloc[i]['date'])
    dst_df.at[i, 'day'] = row_date.strftime('%d') + date_separator + row_date.strftime('%A')
    dst_df.at[i, 'month'] = row_date.strftime('%B')
    dst_df.at[i, 'year'] = int(row_date.strftime('%Y'))

    # calcolo il risultato della partita
    dst_df.at[i, 'outcome'] = return_outcome(dst_df.iloc[i]['home_score'], dst_df.iloc[i]['away_score'])

    # deprecato
    # if(dst_df.iloc[i]['outcome'] == 'D'):
    #     dst_df.at[i, 'outcome'] = find_penalty(data['results'].iloc[i], data['shootouts'], sht_len - 1)

dst_df.drop(labels=['date'], axis=1, inplace=True)
dst_df.to_csv('dataset/dataset.csv')

### To do:
1. Distribuzione dei dati - grafico a barre per anni/decadi
2. PMF sui dati degli anni / approssimaziona a CMF (?)
3. Barplot su:
    - goal fatti
    - sul numero di partite per continente
4. Scatterplot
    - sui goal ?
    - dei risultati di outcome D/H/A (?)

Da vedere come inserire:
- comparare il numero di partite per nazione
- score per games?

Modelli:
    - Regressione logistica
    - Alberi decisionali
    - o...?

Da inserire (?)
    - FIFa ranking per elaborazione modello
        - regole del ranking
        - spiegare funzionamento
        - aggiungere 'ranking' in base alla data
    
Aggiuntivi da implementare via facendo + descrizione
- Eliminare i paesi che hanno cambiato nome/pulizia dataset
- Riduzione delle dimensionalità
    - per il calcolo del modello considera solo se le partite sono state giocate in campo neutrale

In [14]:
dst = pd.read_csv('dataset/dataset.csv')
dst.drop(labels=['Unnamed: 0'], axis=1, inplace=True)

dst.head()

,home_team,away_team,home_score,away_score,tournament,city,country,neutral,outcome,day,month,year
0,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False,draw,"30, Saturday",November,1872
1,England,Scotland,4.0,2.0,Friendly,London,England,False,home,"08, Saturday",March,1873
2,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False,home,"07, Saturday",March,1874
3,England,Scotland,2.0,2.0,Friendly,London,England,False,draw,"06, Saturday",March,1875
4,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False,home,"04, Saturday",March,1876


### Statistica descrittiva - Distribuzione dei dati

Descrizione del dataset. Grafico a barre per anni/decadi